**Decision Variables:**

$$
x_i = 
\begin{cases}
1 & \text{if a plant is installed in city } i \\
0 & \text{otherwise}
\end{cases}
$$

$$
y_{ij} \geq 0 \quad \text{quantity produced at the plant in city } i \text{ and shipped to city } j
$$

---

**Parameters:**

- Set of cities: $\displaystyle N = \{1, 2, \ldots, 10\}$
- Transportation cost per unit from city $ i $ to city $ j $: $\displaystyle C_{ij}$
- Daily installation/operation cost of a plant in city $ i $: $\displaystyle F_i$
- Maximum plant capacity in city $ i $: $\displaystyle Cap_i = 1000$ (for all cities)
- Demand of city $ j $: $\displaystyle D_j$
- Total number of plants to be installed: $\displaystyle \sum_{i \in N} x_i = 7$

---

**Objective Function:**

Minimize the total cost (fixed costs + transportation costs):

$$
\min Z = \sum_{i \in N} F_i x_i \;+\; \sum_{i \in N} \sum_{j \in N} C_{ij} y_{ij}
$$

---

**Constraints:**

1. **Demand Fulfillment:**

For each city $ j $, the entire demand must be met by the sum of the supplies from all plants:

$$
\sum_{i \in N} y_{ij} = D_j \quad \forall j \in N.
$$

2. **Plant Capacity:**

The total quantity sent from the plant in city $ i $ must not exceed the plant's capacity. Moreover, production can only occur if the plant is installed:

$$
\sum_{j \in N} y_{ij} \leq Cap_i \cdot x_i \quad \forall i \in N.
$$

3. **Plant Limit:**

Exactly 7 plants must be installed:

$$
\sum_{i \in N} x_i = 7.
$$

4. **Non-negativity and Binary:**

$$
y_{ij} \geq 0 \quad \forall i,j \in N,
$$

$$
x_i \in \{0,1\} \quad \forall i \in N.
$$

In [6]:
import random
import math

# Data
C = [
[0.00,2.48,3.26,5.54,2.10,2.41,2.50,2.07,6.93,5.35],
[2.48,0.00,0.82,5.96,4.53,3.65,4.70,1.83,7.69,5.18],
[3.26,0.82,0.00,6.06,5.27,4.13,5.36,2.53,7.85,5.11],
[5.54,5.96,6.06,0.00,5.38,3.19,4.55,7.13,1.84,1.41],
[2.10,4.53,5.27,5.38,0.00,2.44,0.91,4.05,6.28,5.73],
[2.41,3.65,4.13,3.19,2.44,0.00,1.87,4.25,4.52,3.31],
[2.50,4.70,5.36,4.55,0.91,1.87,0.00,4.56,5.38,5.03],
[2.07,1.83,2.53,7.13,4.05,4.25,4.56,0.00,8.70,6.60],
[6.93,7.69,7.85,1.84,6.28,4.52,5.38,8.70,0.00,3.15],
[5.35,5.18,5.11,1.41,5.73,3.31,5.03,6.60,3.15,0.00]
]

F = [3904,3147,3960,3253,3097,3954,3008,3469,3840,3508]
D = [400,500,500,350,500,400,200,300,200,450]
Cap = [1000]*10

num_cities = 10
num_plants = 7

def solve_transport_heuristic(install_vector):
    # Transportation Subproblem
    demands = D[:]
    capacities = [Cap[i] if install_vector[i] == 1 else 0 for i in range(num_cities)]

    # Cost List
    cost_list = []
    for i in range(num_cities):
        if install_vector[i] == 1:
            for j in range(num_cities):
                cost_list.append((C[i][j], i, j))
    
    # Sort by Cost
    cost_list.sort(key=lambda x: x[0])

    # Flow Matrix
    y = [[0 for _ in range(num_cities)] for __ in range(num_cities)]

    # Alocation
    for (cost, i, j) in cost_list:
        if capacities[i] > 0 and demands[j] > 0:
            alloc = min(capacities[i], demands[j])
            y[i][j] = alloc
            capacities[i] -= alloc
            demands[j] -= alloc
    
    # Verify
    if any(d > 0 for d in demands):
        return None, None  # não viável

    # Total Cost
    fixed_cost = sum(F[i]*install_vector[i] for i in range(num_cities))
    transport_cost = 0
    for i in range(num_cities):
        for j in range(num_cities):
            transport_cost += C[i][j]*y[i][j]

    total_cost = fixed_cost + transport_cost
    return total_cost, y

def generate_initial_solution():
    chosen = random.sample(range(num_cities), num_plants)
    install_vector = [0]*num_cities
    for c in chosen:
        install_vector[c] = 1
    return install_vector

def neighbor_solution(install_vector):
    installed = [i for i in range(num_cities) if install_vector[i] == 1]
    not_installed = [i for i in range(num_cities) if install_vector[i] == 0]

    city_to_remove = random.choice(installed)
    city_to_add = random.choice(not_installed)

    new_install = install_vector[:]
    new_install[city_to_remove] = 0
    new_install[city_to_add] = 1
    return new_install

# SA Params
max_iterations = 1000
initial_temp = 1000.0
cooling_rate = 0.99

random.seed(42)  # Reproductibility

current_solution = generate_initial_solution()
current_cost, _ = solve_transport_heuristic(current_solution)

# Try More Times If Error
for _ in range(10):
    if current_cost is not None:
        break
    current_solution = generate_initial_solution()
    current_cost, _ = solve_transport_heuristic(current_solution)

if current_cost is None:
    # No Solution
    raise RuntimeError("Não foi encontrada uma solução inicial viável.")

best_solution = current_solution[:]
best_cost = current_cost

temperature = initial_temp

for iteration in range(max_iterations):
    candidate_solution = neighbor_solution(current_solution)
    candidate_cost, _ = solve_transport_heuristic(candidate_solution)

    if candidate_cost is None:
        # Skip
        continue

    cost_diff = candidate_cost - current_cost

    if cost_diff < 0:
        # Accept Solution
        current_solution = candidate_solution
        current_cost = candidate_cost
        if candidate_cost < best_cost:
            best_solution = candidate_solution[:]
            best_cost = candidate_cost
    else:
        # Accept Probability exp(-cost_diff/temperature)
        prob = math.exp(-cost_diff / temperature)
        if random.random() < prob:
            current_solution = candidate_solution
            current_cost = candidate_cost

    # Cooling
    temperature *= cooling_rate

final_cost, final_y = solve_transport_heuristic(best_solution)

print("Melhor solução encontrada:", best_solution)
print("Cidades com plantas:", [i+1 for i in range(num_cities) if best_solution[i] == 1])
print("Custo da melhor solução:", best_cost)
print("Matriz de fluxo (y_ij):")
for row in final_y:
    print(row)

Melhor solução encontrada: [1, 1, 0, 1, 1, 0, 1, 1, 0, 1]
Cidades com plantas: [1, 2, 4, 5, 7, 8, 10]
Custo da melhor solução: 24912.0
Matriz de fluxo (y_ij):
[400, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 500, 500, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 350, 0, 0, 0, 0, 200, 0]
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 400, 200, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 300, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 450]


In [7]:
import numpy as np

np.sum(final_y, axis=0)

array([400, 500, 500, 350, 500, 400, 200, 300, 200, 450])

In [8]:
np.sum(final_y, axis=1)

array([ 400, 1000,    0,  550,  500,    0,  600,  300,    0,  450])